In [ ]:
import sys
from google.colab import drive
drive.mount('/content/gdrive') 
sys.path.append('/content/gdrive/MyDrive/My_env')
# ______________________________________________________________________

import os
import pandas as pd
import numpy as np
import pickle
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')

# Modeling
from sklearn.model_selection import StratifiedKFold

from sklearn.metrics import accuracy_score
# CAT Classifier
# from catboost import CatBoostClassifier
# CAT Regressor 
# from catboost import CatBoostRegressor

In [ ]:
""" To Classifier """
def to_binary(pred) :
  pred_conv = pred.copy()
  if type(pred) == list :
    pred_conv = np.array(pred_conv)
  pred_conv[pred_conv >= 0.5] = 1
  pred_conv[pred_conv < 0.5] = 0
  
  return pred_conv

In [ ]:
contest = ""
dir = "/content/gdrive/MyDrive/ColabNotes/DACON/{}/".format(contest)
data_dir = dir + "data/"
submission_file = "sample_submission.csv"
test_file = "test_renew.csv"
X_train_file = "X_train.csv"
Y_train_file = "Y_train.csv"

submission = pd.read_csv(data_dir + submission_file)
test = pd.read_csv(data_dir + test_file)
X_train = pd.read_csv(data_dir + X_train_file)
Y_train = pd.read_csv(data_dir + Y_train_file)

In [ ]:
rows_train = X_train.shape[0] # 주어진 train data의 row 수
rows_test = test.shape[0] # 주어진 test data의 row 수
splits_tr = 10 # 모델 트레이닝을 진행할 때의 kfold 수
num_seed_tr = 10 # 트레이닝 seed 개수

pred_dict = {}
pred_test_dict = {}

## Modeling with CATboost

In [ ]:
""" 예측 성능 지표 NMAE """
def NMAE(true, pred) :
  score = np.mean(np.abs(true - pred) / true)
  
  return score

In [ ]:
rand_seeds = np.random.randint(0, 1000, num_seed_tr)

for i, seed in enumerate(rand_seeds) : 

  kfold = StratifiedKFold(n_splits = splits_tr, random_state = seed, shuffle = True)
  cv_pred = np.zeros(rows_train)
  pred_test = np.zeros(rows_test)

  for n, (train_idx, val_idx) in enumerate(kfold.split(X_train, Y_train)) :
        
        x_tr, x_val = X_train.iloc[train_idx], X_train.iloc[val_idx]
        y_tr, y_val = Y_train.iloc[train_idx].values.ravel(), Y_train.iloc[val_idx].values.ravel()
        
        """ CAT Classifier """
        #catmodel = CatBoostClassifier(iterations = 10000, task_type = "GPU")
        """ CAT Regressor """
        #catmodel = CatBoostRegressor(iterations = 10000, task_type = "GPU")

        catmodel.fit(x_tr, y_tr, eval_set = [(x_val, y_val)], early_stopping_rounds = 50, verbose = False)

        pred = catmodel.predict(x_val)
        cv_pred[val_idx] = pred
        pred_test += catmodel.predict(test) / splits_tr

  pred_dict['cat' + str(seed)] = cv_pred
  pred_test_dict['cat' + str(seed)] = pred_test
  
  """ CAT Classifier"""
  print(f'seed {seed}', 'Acc :', accuracy_score(Y_train, to_binary(cv_pred)))
  
  """ CAT Regressor """
  #print(f'seed {seed}', 'Acc :', accuracy_score(Y_train, cv_pred))


In [ ]:
""" NMAE로 측정할 경우 """
print(f'seed {seed}', 'NMAE :', NMAE(Y_train, cv_pred))

In [ ]:
val_result_df = pd.DataFrame(pred_dict)
val_result = []
for ii in range(len(val_result_df)) :
  single_mean = np.mean(val_result_df.iloc[ii, :])
  val_result.append(single_mean)

""" Classifier인 경우 """
#val_result = to_binary(val_result)

print("최종 모델 Accuracy : ", accuracy_score(Y_train, val_result))

In [ ]:
model = "cat"

pred_train_df = pd.DataFrame(pred_dict)
pred_train_df = pred_train_df[np.sort(pred_train_df.columns)]

with open(dir + "pickle/pred_train_" + model + ".pickle", "wb") as ptr :
  pickle.dump(pred_train_df, ptr)

pred_test_df = pd.DataFrame(pred_test_dict)
pred_test_df = pred_test_df[np.sort(pred_test_df.columns)]

with open(dir + "pickle/pred_test_" + model + ".pickle", "wb") as pte :
  pickle.dump(pred_test_df, pte)

## Submission

In [ ]:
with open(dir + "pickle/pred_test_cat.pickle", 'rb') as pte :
    pred_test_df = pickle.load(pte)

In [ ]:
pred_test_df

In [ ]:
final_result = []
for ii in range(len(pred_test_df)) :
  single_mean = np.mean(pred_test_df.iloc[ii, :])
  final_result.append(single_mean)

""" Classifier인 경우 """
#final_result = to_binary(final_result)

final_result

In [ ]:
submission["target"] = final_result

#Classierfier
#submission = submission.astype(int)

submission

In [ ]:
save_name = ""
submission.to_csv(dir + "data/results/{}.csv".format(save_name), index = False)
sns.countplot(x = submission["target"])